In [1]:
import numpy as np
print("numpy version: {}".format(np.__version__))
import pandas as pd 
print("pandas version: {}".format(pd.__version__))
import matplotlib
import matplotlib.pyplot as plt
print("matplotlib version: {}".format(matplotlib.__version__))
import scipy as sp
print("scipy version: {}".format(sp.__version__))
import sklearn as sl
print("scikit-learn: {}".format(sl.__version__))
import seaborn as sns
print("seaborn: {}".format(sns.__version__))
import statsmodels as sm
print("statsmodels: {}".format(sm.__version__))

numpy version: 1.17.4
pandas version: 0.25.3
matplotlib version: 3.1.2
scipy version: 1.3.3
scikit-learn: 0.21.3
seaborn: 0.9.0
statsmodels: 0.10.2


# GroupBy Mechanics

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows ( ```axis=0``` ) or its columns ( ```axis=1``` ). Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what’s being done to the data.

![title](images/group_aggregation.png)

Each grouping key can take many forms, and the keys do not have to be all of the same type:

- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dict or series giving a correspondence between the values on the axis being grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                    'key2' : ['one', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,0.157596,1.015309
1,a,two,-0.476093,-1.270983
2,b,one,1.451185,-1.335462
3,b,two,-0.849053,-0.249643
4,a,one,-0.582127,-1.326254


Suppose you wanted to compute the mean of the data1 column using the labels from
key1 . There are a number of ways to do this. One is to access data1 and call groupby
with the column (a Series) at key1 :

In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

The idea is that this object has all of the information needed to then apply some operation to each of the groups.

In [6]:
grouped.mean()

key1
a   -0.300208
b    0.301066
Name: data1, dtype: float64

the data (a Series) has been aggregated according to the group key,
producing a new Series that is now indexed by the unique values in the key1 column.

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -0.212265
      two    -0.476093
b     one     1.451185
      two    -0.849053
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,-0.212265,-0.476093
b,1.451185,-0.849053


In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.476093
            2006    1.451185
Ohio        2005   -0.345728
            2006   -0.582127
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.300208,-0.527309
b,0.301066,-0.792553


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.212265 -0.155473
     two  -0.476093 -1.270983
b    one   1.451185 -1.335462
     two  -0.849053 -0.249643

Regardless of the objective in using ```groupby``` , a generally useful GroupBy method is
```size``` , which returns a Series containing group sizes(any missing values in a group key will be excluded from the result.):

In [15]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print()

a
  key1 key2     data1     data2
0    a  one  0.157596  1.015309
1    a  two -0.476093 -1.270983
4    a  one -0.582127 -1.326254

b
  key1 key2     data1     data2
2    b  one  1.451185 -1.335462
3    b  two -0.849053 -0.249643



In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
    print()

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.157596  1.015309
4    a  one -0.582127 -1.326254

('a', 'two')
  key1 key2     data1     data2
1    a  two -0.476093 -1.270983

('b', 'one')
  key1 key2     data1     data2
2    b  one  1.451185 -1.335462

('b', 'two')
  key1 key2     data1     data2
3    b  two -0.849053 -0.249643



In [18]:
pieces = dict(list(df.groupby('key1')))

In [19]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.451185,-1.335462
3,b,two,-0.849053,-0.249643


By default groupby groups on ```axis=0``` , but you can group on any of the other axes.

In [20]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)

In [22]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.157596  1.015309
1 -0.476093 -1.270983
2  1.451185 -1.335462
3 -0.849053 -0.249643
4 -0.582127 -1.326254
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array
of column names has the effect of column subsetting for aggregation. This means
that:
```python
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
are syntactic sugar for:
```python
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```
Especially for large datasets, it may be desirable to aggregate only a few columns. For
example, in the preceding dataset, to compute means for just the data2 column and
get the result as a DataFrame, we could write:

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.155473
     two  -1.270983
b    one  -1.335462
     two  -0.249643

In [24]:
df

,key1,key2,data1,data2
0,a,one,0.157596,1.015309
1,a,two,-0.476093,-1.270983
2,b,one,1.451185,-1.335462
3,b,two,-0.849053,-0.249643
4,a,one,-0.582127,-1.326254


The object returned by this indexing operation is a grouped DataFrame if a list or
array is passed or a grouped Series if only a single column name is passed as a scalar:

In [25]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [26]:
s_grouped

In [27]:
s_grouped.mean()

key1  key2
a     one    -0.155473
      two    -1.270983
b     one    -1.335462
      two    -0.249643
Name: data2, dtype: float64

In [28]:
s_grouped.sum()

key1  key2
a     one    -0.310945
      two    -1.270983
b     one    -1.335462
      two    -0.249643
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [29]:
people = pd.DataFrame(np.random.randn(5, 5),
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [30]:
people.iloc[2:3, [1, 2]] = np.nan

In [31]:
people

,a,b,c,d,e
Joe,0.677257,-0.887433,0.775734,0.248496,-0.305271
Steve,-1.714727,-0.050279,-0.092963,1.070039,1.536346
Wes,-0.479745,NaN,NaN,1.419943,-1.497247
Jim,-0.117587,-0.183726,0.329227,-0.548810,-1.994652
Travis,-0.371253,0.144365,0.543344,0.089948,1.319310


In [32]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [33]:
by_column = people.groupby(mapping, axis=1)

In [34]:
by_column.sum()

,blue,red
Joe,1.024231,-0.515447
Steve,0.977076,-0.228659
Wes,1.419943,-1.976992
Jim,-0.219584,-2.295965
Travis,0.633293,1.092422


In [35]:
map_series = pd.Series(mapping)

In [36]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [37]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

Using Python functions is a more generic way of defining a group mapping compared
with a dict or Series. Any function passed as a group key will be called once per index
value, with the return values being used as the group names. More concretely, consider the example DataFrame from the previous section, which has people’s first
names as index values. Suppose you wanted to group by the length of the names;
while you could compute an array of string lengths, it’s simpler to just pass the len
function:

In [38]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.079925,-1.071158,1.104961,1.119629,-3.797170
5,-1.714727,-0.050279,-0.092963,1.070039,1.536346
6,-0.371253,0.144365,0.543344,0.089948,1.319310


In [39]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [40]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.479745 -0.887433  0.775734  0.248496 -1.497247
  two -0.117587 -0.183726  0.329227 -0.548810 -1.994652
5 one -1.714727 -0.050279 -0.092963  1.070039  1.536346
6 two -0.371253  0.144365  0.543344  0.089948  1.319310

### Grouping by Index Levels

In [41]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

In [42]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [43]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.269108  0.283176  0.850843  0.195998  0.391296
1     -0.892037 -1.137726  0.326113 -0.552480  0.834823
2      0.341512 -1.691804 -0.273038  1.488221 -0.325883
3     -0.894907 -0.636833  0.748760 -1.648343  2.064781

In [44]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [45]:
hier_df.groupby(level='tenor', axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


In [46]:
hier_df.groupby(by=[lambda x: abs(x) > 0]).count()

cty   US       JP   
tenor  1  3  5  1  3
False  1  1  1  1  1
True   3  3  3  3  3

# Data Aggregation

**Aggregations** refer to any data transformation that produces scalar values from arrays.

- mean, median
- count
- min, max
- std, var
- sum, prod
- first, last

While **quantile** is not explicitly implemented for GroupBy, it is a Series method and
thus available for use. Internally, GroupBy efficiently slices up the Series, calls
```piece.quantile(0.9)``` for each piece, and then assembles those results together into
the result object:

In [47]:
df

,key1,key2,data1,data2
0,a,one,0.157596,1.015309
1,a,two,-0.476093,-1.270983
2,b,one,1.451185,-1.335462
3,b,two,-0.849053,-0.249643
4,a,one,-0.582127,-1.326254


In [48]:
grouped = df.groupby('key1')

In [49]:
grouped['data1']

In [50]:
grouped['data1'].quantile(0.9)

key1
a    0.030858
b    1.221161
Name: data1, dtype: float64

In [51]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [52]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.739723,2.341562
b,2.300238,1.085819


In [53]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.300208  0.399999 -0.582127 -0.529110 -0.476093 -0.159249   
b      2.0  0.301066  1.626514 -0.849053 -0.273993  0.301066  0.876126   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.157596   3.0 -0.527309  1.336232 -1.326254 -1.298618 -1.270983   
b     1.451185   2.0 -0.792553  0.767790 -1.335462 -1.064007 -0.792553   

                          
           75%       max  
key1                      
a    -0.127837  1.015309  
b    -0.521098 -0.249643

## Column-Wise and Multiple Function Application

In [54]:
tips = pd.read_csv('examples/tips.csv')

*Add tip percentage of total bill*

In [55]:
tips['tips_pct'] = tips['tip'] / tips['total_bill']

In [56]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tips_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [57]:
grouped = tips.groupby(['day', 'smoker'])

In [58]:
grouped_pct = grouped['tips_pct']

In [59]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [60]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

pass a list of (name, function) tuples, the first element of each tuple will be used as
the DataFrame column names (you can think of a list of 2-tuples as an ordered
mapping):

In [61]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [62]:
functions = ['count', 'mean', 'max']

In [63]:
result = grouped['tips_pct', 'total_bill'].agg(functions)

In [64]:
result

tips_pct                     total_bill                  
               count      mean       max      count       mean    max
day  smoker                                                          
Fri  No            4  0.151650  0.187735          4  18.420000  22.75
     Yes          15  0.174783  0.263480         15  16.813333  40.17
Sat  No           45  0.158048  0.291990         45  19.661778  48.33
     Yes          42  0.147906  0.325733         42  21.276667  50.81
Sun  No           57  0.160113  0.252672         57  20.506667  48.17
     Yes          19  0.187250  0.710345         19  24.120000  45.35
Thur No           45  0.160298  0.266312         45  17.113111  41.19
     Yes          17  0.163863  0.241255         17  19.190588  43.11

The resulting DataFrame has hierarchical columns, the same as you
would get aggregating each column separately and using concat to glue the results
together using the column names as the keys argument:

In [65]:
result['tips_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [66]:
ftuples = [('Durchshnitt', 'mean'), ('Abweichung', np.var)]

In [67]:
grouped['tips_pct', 'total_bill'].agg(ftuples)

tips_pct             total_bill            
            Durchshnitt Abweichung Durchshnitt  Abweichung
day  smoker                                               
Fri  No        0.151650   0.000791   18.420000   25.596333
     Yes       0.174783   0.002631   16.813333   82.562438
Sat  No        0.158048   0.001581   19.661778   79.908965
     Yes       0.147906   0.003767   21.276667  101.387535
Sun  No        0.160113   0.001793   20.506667   66.099980
     Yes       0.187250   0.023757   24.120000  109.046044
Thur No        0.160298   0.001503   17.113111   59.625081
     Yes       0.163863   0.001551   19.190588   69.808518

suppose you wanted to apply potentially different functions to one or more of
the columns. To do this, pass a dict to ```agg``` that contains a mapping of column names
to any of the function specifications listed so far:

In [68]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [69]:
grouped.agg({'tips_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tips_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

A DataFrame will have hierarchical columns only if multiple functions are applied to
at least one column.

## Returning Aggregated Data Without Row Indexes

In all of the examples up until now, the aggregated data comes back with an index,
potentially hierarchical, composed from the unique group key combinations. Since
this isn’t always desirable, you can disable this behavior in most cases by passing
```as_index=False``` to ```groupby``` :

In [70]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tips_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [71]:
result.reset_index()

day smoker tips_pct                     total_bill                  
                  count      mean       max      count       mean    max
0   Fri     No        4  0.151650  0.187735          4  18.420000  22.75
1   Fri    Yes       15  0.174783  0.263480         15  16.813333  40.17
2   Sat     No       45  0.158048  0.291990         45  19.661778  48.33
3   Sat    Yes       42  0.147906  0.325733         42  21.276667  50.81
4   Sun     No       57  0.160113  0.252672         57  20.506667  48.17
5   Sun    Yes       19  0.187250  0.710345         19  24.120000  45.35
6  Thur     No       45  0.160298  0.266312         45  17.113111  41.19
7  Thur    Yes       17  0.163863  0.241255         17  19.190588  43.11

Of course, it’s always possible to obtain the result in this format by calling
```reset_index``` on the result. Using the ```as_index=False``` method avoids some unnecessary computations.

## Apply: General split-apply-combine

The most general-purpose GroupBy method is ```apply```

![title](images/group_aggregation.png)

suppose you wanted to select the top five tip_pct values by group. First, write a function that selects the rows with the largest values in a particular column:

In [72]:
def top(df, n=5, column='tips_pct'):
    return df.sort_values(by=column)[-n:]

In [73]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tips_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


if we group by smoker , say, and call apply with this function, we get the
following

In [74]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size  tips_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

The ```top``` function is called on each row group from the DataFrame, and 
then the results are glued together using ```pandas.concat``` , labeling the
pieces with the group names.

If you pass a function to ```apply``` that takes other arguments or keywords, you can pass
these after the function:

In [75]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size  tips_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [76]:
result = tips.groupby('smoker')['tips_pct'].describe()

In [77]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [78]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [79]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when you invoke a method like ```describe``` , it is actually just a shortcut for

In [80]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size  tips_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys

disable indexes this by passing ```group_keys=False``` to ```groupby``` :

In [81]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tips_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [82]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [83]:
quartiles = pd.cut(frame.data1, 4)

In [84]:
quartiles[:10]

0     (-0.105, 1.648]
1     (-0.105, 1.648]
2     (-0.105, 1.648]
3     (-0.105, 1.648]
4    (-1.858, -0.105]
5     (-0.105, 1.648]
6    (-1.858, -0.105]
7    (-1.858, -0.105]
8    (-1.858, -0.105]
9    (-1.858, -0.105]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.617, -1.858] < (-1.858, -0.105] < (-0.105, 1.648] < (1.648, 3.401]]

The ```Categorical``` object returned by ```cut``` can be passed directly to ```groupby```.
So we could compute a set of statistics for the data2 column like so:

In [85]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [86]:
grouped = frame.data2.groupby(quartiles)

In [87]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.617, -1.858]",-1.742408,2.067400,36.0,0.229906
"(-1.858, -0.105]",-2.662722,2.453043,411.0,-0.058364
"(-0.105, 1.648]",-2.867093,2.973326,503.0,-0.033423
"(1.648, 3.401]",-2.201432,1.726680,50.0,-0.079367


return quantile numbers

In [88]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [89]:
grouped = frame.data2.groupby(grouping)

In [90]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.364267,2.143875,100.0,0.229996
1,-2.662722,2.302768,100.0,-0.134153
2,-2.157793,1.691309,100.0,-0.151360
3,-2.502043,2.453043,100.0,-0.072783
4,-2.630456,2.056720,100.0,0.016257
5,-2.067018,2.670303,100.0,0.107450
6,-2.806231,2.625341,100.0,-0.105263
7,-2.609557,2.604515,100.0,-0.071512
8,-2.867093,2.421991,100.0,-0.136632


In [91]:
grouping

0      6
1      6
2      8
3      5
4      0
      ..
995    0
996    6
997    3
998    2
999    9
Name: data1, Length: 1000, dtype: int64

In [92]:
grouping.unique()

array([6, 8, 5, 0, 3, 7, 2, 1, 9, 4])

### Example: Filling Missing Values with Group-Specific Values

When cleaning up missing data, in some cases you will replace data observations
using ```dropna```, but in others you may want to impute (fill in) the null (NA) values
using a fixed value or some value derived from the data. ```fillna``` is the right tool to
use; for example, here I fill in NA values with the mean:

In [93]:
s = pd.Series(np.random.randn(6))

In [94]:
s[::2] = np.nan

In [95]:
s

0         NaN
1   -0.908020
2         NaN
3   -0.067034
4         NaN
5   -0.478656
dtype: float64

In [96]:
s.fillna(s.mean())

0   -0.484570
1   -0.908020
2   -0.484570
3   -0.067034
4   -0.484570
5   -0.478656
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the
data and use ```apply``` with a function that calls ```fillna``` on each data chunk. Here is some sample data on US states divided into eastern and western regions:

In [97]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 
          'Oregon', 'Nevada', 'California', 'Idaho']

In [98]:
group_key = ['East'] * 4 + ['West'] * 4

In [99]:
data = pd.Series(np.random.randn(8), index=states)

In [100]:
data

Ohio         -1.437575
New York      0.514658
Vermont       0.521154
Florida       1.308435
Oregon        0.604890
Nevada        2.486189
California   -0.110510
Idaho         1.633221
dtype: float64

In [101]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [102]:
data

Ohio         -1.437575
New York      0.514658
Vermont            NaN
Florida       1.308435
Oregon        0.604890
Nevada             NaN
California   -0.110510
Idaho              NaN
dtype: float64

In [103]:
data.groupby(group_key).mean()

East    0.128506
West    0.247190
dtype: float64

In [104]:
fill_mean = lambda g: g.fillna(g.mean())

In [105]:
data.groupby(group_key).apply(fill_mean)

Ohio         -1.437575
New York      0.514658
Vermont       0.128506
Florida       1.308435
Oregon        0.604890
Nevada        0.247190
California   -0.110510
Idaho         0.247190
dtype: float64

In [106]:
fill_values = {'East': 0.5, 'West': -1}

In [107]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [108]:
data.groupby(group_key).apply(fill_func)

Ohio         -1.437575
New York      0.514658
Vermont       0.500000
Florida       1.308435
Oregon        0.604890
Nevada       -1.000000
California   -0.110510
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

*Hearts, Spades, Clubs, Diamonds*

In [109]:
suits = ['H', 'S', 'C', 'D']

In [110]:
card_val = (list(range(1, 11)) + [10] * 3) * 4

In [111]:
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

In [112]:
cards = []

In [113]:
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

In [114]:
deck = pd.Series(card_val, index=cards)

In [115]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [116]:
deck is iter

False

In [117]:
def draw(deck, n=5):
    return deck.sample(n)

In [118]:
draw(deck)

7S     7
3H     3
2S     2
QC    10
9H     9
dtype: int64

In [119]:
get_suit = lambda card: card[-1]

In [120]:
deck.groupby(get_suit).apply(draw, n=2)

C  JC     10
   5C      5
D  KD     10
   10D    10
H  7H      7
   3H      3
S  QS     10
   9S      9
dtype: int64

In [121]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

4C      4
5C      5
10D    10
8D      8
QH     10
3H      3
AS      1
7S      7
dtype: int64

### Example: Group Weighted Average and Correlation

Under the split-combine paradigm of ```groupby```, operations between columns in a DataFrame
or two Series, such as a group weighted average, are possible. As an example, take this dataset
containing group keys, values, and some weights:

In [130]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                                'data': np.random.randn(8),
                                'weights': np.random.rand(8)})

In [131]:
df

,category,data,weights
0,a,-1.006089,0.121424
1,a,1.205551,0.375222
2,a,0.224113,0.771084
3,a,0.892098,0.050248
4,b,-0.765133,0.822962
5,b,0.257090,0.814412
6,b,-0.598180,0.804014
7,b,-2.086167,0.535509


In [132]:
grouped = df.groupby('category')

In [133]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [134]:
grouped.apply(get_wavg)

category
a    0.415653
b   -0.678023
dtype: float64

In [135]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [136]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [137]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with SPX . As one way to do
this, we first create a function that computes the pairwise correlation of each column
with the 'SPX' column:

In [138]:
spx_corr = lambda x: x.corrwith(x['SPX'])

Next, we compute percent change on ```close_px``` using ```pct_change```:

In [139]:
rets = close_px.pct_change().dropna()

Lastly, we group these percent changes by year, which can be extracted from each row
label with a one-line function that returns the year attribute of each datetime label:

In [140]:
get_year = lambda x: x.year

In [141]:
by_year = rets.groupby(get_year)

In [143]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


You could also compute inter-column correlations. Here we compute the annual correlation between Apple and Microsoft:

In [144]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [145]:
close_px.columns

Index(['AAPL', 'MSFT', 'XOM', 'SPX'], dtype='object')

In [146]:
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [175]:
get_month = lambda x: x.month

In [176]:
get_year = lambda x: x.year

In [177]:
by_year_month = rets.groupby([get_month, get_year])

In [181]:
by_year_month.agg(['sum', 'min', 'max'])

AAPL                          MSFT                           XOM  \
              sum       min       max       sum       min       max       sum   
1  2003 -0.026482 -0.041499  0.031117 -0.117327 -0.070345  0.029094 -0.034112   
   2004  0.058643 -0.055372  0.042293  0.011477 -0.019196  0.025276 -0.003980   
   2005  0.185726 -0.063805  0.072491 -0.016104 -0.018963  0.013626  0.007596   
   2006  0.056937 -0.041823  0.063248  0.075527 -0.022690  0.048801  0.113325   
   2007  0.018368 -0.061927  0.083070  0.034263 -0.020502  0.035198 -0.031432   
...           ...       ...       ...       ...       ...       ...       ...   
12 2006 -0.073458 -0.031059  0.049091  0.017384 -0.011364  0.019048 -0.000767   
   2007  0.087006 -0.031462  0.035789  0.060734 -0.026098  0.042011  0.051157   
   2008 -0.070973 -0.065703  0.060851 -0.027362 -0.079516  0.057311  0.004839   
   2009  0.055631 -0.022605  0.041765  0.036587 -0.016586  0.025650 -0.094775   
   2010  0.036372 -0.006060  0.016873  0.101184 -0.008279  0.032473  0.050388   

                                  SPX                      
              min       max       sum       min       max  
1  2003 -0.033712  0.036431 -0.058415 -0.029233  0.022474  
   2004 -0.016515  0.023249  0.017618 -0.013610  0.012395  
   2005 -0.022717  0.014801 -0.025204 -0.011671  0.009675  
   2006 -0.015803  0.041046  0.025636 -0.018326  0.016430  
   2007 -0.032981  0.023583  0.014187 -0.011270  0.008501  
...           ...       ...       ...       ...       ...  
12 2006 -0.023089  0.019524  0.012715 -0.005316  0.008885  
   2007 -0.018960  0.020418 -0.007488 -0.025271  0.016670  
   2008 -0.072849  0.044467  0.018008 -0.089295  0.051360  
   2009 -0.043054  0.012808  0.018081 -0.011811  0.012075  
   2010 -0.004652  0.025457  0.063668 -0.005122  0.021617  

[106 rows x 12 columns]

In [182]:
by_year_month

In [183]:
by_year_month.apply(spx_corr)

AAPL      MSFT       XOM  SPX
1  2003  0.625648  0.688178  0.806282  1.0
   2004  0.459399  0.787128  0.735095  1.0
   2005  0.136962  0.534605  0.711886  1.0
   2006  0.350912  0.671866  0.696231  1.0
   2007  0.153706  0.746843  0.413104  1.0
...           ...       ...       ...  ...
12 2006  0.152562  0.686793  0.564786  1.0
   2007  0.817527  0.806107  0.894348  1.0
   2008  0.649271  0.931241  0.853825  1.0
   2009  0.301882  0.728243  0.207092  1.0
   2010  0.662493  0.756466  0.779365  1.0

[106 rows x 4 columns]

### Example: Group-Wise Linear Regression

In the same theme as the previous example, you can use ```groupby``` to perform more
complex group-wise statistical analysis, as long as the function returns a pandas
object or scalar value. For example, I can define the following regress function
(using the statsmodels econometrics library), which executes an ordinary least
squares (OLS) regression on each chunk of data:

In [184]:
import statsmodels.api as sm

In [186]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

Now, to run a yearly linear regression of AAPL on SPX returns, execute:

In [187]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### Pivot Tables and Cross-Tabulation

A **pivot table** is a data summarization tool frequently found in spreadsheet programs
and other data analysis software. It aggregates a table of data by one or more keys,
arranging the data in a rectangle with some of the group keys along the rows and
some along the columns. Pivot tables in Python with pandas are made possible
through the groupby facility described in this chapter combined with reshape operations utilizing hierarchical indexing. DataFrame has a ```pivot_table``` method, and
there is also a top-level ```pandas.pivot_table``` function. In addition to providing a
convenience interface to ```groupby``` , ```pivot_table``` can add partial totals, also known as **margins**.

suppose you wanted to compute a table of group means (the default pivot_table aggregation type) arranged by day and smoker on the rows:

In [188]:
tips.pivot_table(index=['day', 'smoker'])

size       tip  tips_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

This could have been produced with ```groupby``` directly. Now, suppose we want to
aggregate only tips_pct and size , and additionally group by time . I’ll put smoker in
the table columns and day in the rows:

In [189]:
tips.pivot_table(['tips_pct', 'size'], index=['time', 'day'], columns='smoker')

size            tips_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We could augment this table to include partial totals by passing ```margins=True```. This
has the effect of adding All row and column labels, with corresponding values being
the group statistics for all the data within a single tier:

In [190]:
tips.pivot_table(['tips_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', margins=True)

size                      tips_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

To use a different aggregation function, pass it to ```aggfunc``` . For example, 'count' or
len will give you a cross-tabulation (count or frequency) of group sizes:

In [191]:
tips.pivot_table('tips_pct', index=['time', 'smoker'], 
                 columns='day', margins=True, aggfunc=len)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

### Cross-Tabulations: Crosstab

A **cross-tabulation** (or *crosstab* for short) is a special case of a pivot table that computes group frequencies.